In [1]:
# Libraries
import sys
sys.path.append('./models/CNN_6.py')
import numpy as np
import torch
from torchsummary import summary
from pytorch2keras.converter import pytorch_to_keras

# Check does your computer support using GPU
print("GPU_available={}".format(torch.cuda.is_available()))
torch.device('cuda')


GPU_available=True


device(type='cuda')

In [2]:
from models.CNN_6 import Network as CNN_6

# Instantiate an instance of your custom model
model = CNN_6(num_classes=2)
model = model.cuda() #move the model to the GPU

# Load the saved model state from the .pth file
state_dict = torch.load('./data/Arch6/CNN.pth')

# Load the model state into the custom model
model.load_state_dict(state_dict)

summary(model, input_size=(1,22,500))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 100, 22, 500]           1,000
              ReLU-2         [-1, 100, 22, 500]               0
         MaxPool2d-3         [-1, 100, 11, 250]               0
           Dropout-4         [-1, 100, 11, 250]               0
            Conv2d-5         [-1, 100, 11, 250]          90,100
         MaxPool2d-6          [-1, 100, 5, 125]               0
           Dropout-7          [-1, 100, 5, 125]               0
            Conv2d-8          [-1, 300, 6, 125]         180,300
         MaxPool2d-9           [-1, 300, 3, 62]               0
          Dropout-10           [-1, 300, 3, 62]               0
           Conv2d-11           [-1, 300, 5, 58]         630,300
        MaxPool2d-12           [-1, 300, 3, 29]               0
          Dropout-13           [-1, 300, 3, 29]               0
           Conv2d-14           [-1, 100

c:\Users\Abdulsatar\Documents\UBICOMP\Thesis Work\Cyber Sickness\models\CNN_6.py:80: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax13(x)


In [3]:
# Convert PyTorch -> Onnx -> Keras
import torch.onnx
import onnx
from onnx2keras import onnx_to_keras

input_var = torch.zeros((1, 1, 22, 500))
input_var = input_var.cuda()

torch.onnx.export(model, input_var, 'model.onnx', verbose=True)

onnx_model = onnx.load("model.onnx")
input_all = [node.name for node in onnx_model.graph.input]
print(input_all)


c:\Users\Abdulsatar\.conda\envs\Image_Reconstruction\lib\site-packages\torch\nn\modules\module.py:1194: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return forward_call(*input, **kwargs)


['input.1']


In [5]:
# Conver onnx model to keras model
k_model = onnx_to_keras(onnx_model, input_all, verbose=True, name_policy='renumerate')

# Save the Keras model as an H5 file
k_model.save("data/Arch6/CNN.h5")

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.1.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> 50.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight conv1.weight with shape (100, 1, 3, 3).
DEBUG:onnx2keras:Found weight conv1.bias with shape (100,).
DEBUG:onnx2keras:Found weight conv2.weight with shape (100, 100, 3, 3).
DEBUG:onnx2keras:Found weight conv2.bias with shape (100,).
DEBUG:onnx2keras:Found weight conv3.weight with shape (300, 100, 2, 3).
DEBUG:onnx2keras:Found weight conv3.bias with shape (300,).
DEBUG:onnx2keras:Found weight conv4.weight with shape (300, 300, 1, 7).
DEBUG:onnx2keras:Found weight conv4.bias with shape (300,).
DEBUG:onnx2keras:Found weight conv5.weight with shape (100, 300, 1, 3).
DEBUG:onnx2keras:Found weight conv5.bias with shape (100,).
DEBUG:onnx2keras:Found weight conv6.weight with shape (100, 100, 1

DEBUG:h5py._conv:Creating converter from 5 to 3
